In [2]:
import os

In [3]:
%pwd

'/Users/armandoreyesmartinez/00_End_to_End_ML/Chicken_Disease_Classification_Project/research'

In [4]:
os.chdir('../')

In [5]:
os.getcwd()

'/Users/armandoreyesmartinez/00_End_to_End_ML/Chicken_Disease_Classification_Project'

In [6]:
%pwd

'/Users/armandoreyesmartinez/00_End_to_End_ML/Chicken_Disease_Classification_Project'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [8]:
## This is a unit test that need to be improved
import unittest

# Compatibility for different Python versions
if not hasattr(unittest.TestCase, 'assertRaisesRegexp'):
    unittest.TestCase.assertRaisesRegexp = unittest.TestCase.assertRaisesRegex


from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import logger


In [9]:
from box import ConfigBox
from box.exceptions import BoxValueError

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        try:
            # Verify files exist
            if not os.path.exists(config_filepath):
                raise FileNotFoundError(f"Config file not found at {config_filepath}")
            if not os.path.exists(params_filepath):
                raise FileNotFoundError(f"Params file not found at {params_filepath}")
                
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            
            # Verify required fields
            if not self.config.get('artifacts_root'):
                raise ValueError("artifacts_root not found in config")
                
            create_directories([self.config.artifacts_root])
            
        except BoxValueError as e:
            logger.error(f"Configuration error in YAML parsing: {e}")
            raise
        except Exception as e:
            logger.error(f"Error in configuration: {str(e)}")
            raise


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
config = read_yaml(CONFIG_FILE_PATH)

In [13]:
print(config.artifacts_root)

artifacts


In [14]:
print(config.data_ingestion.root_dir)

artifacts/data_ingestion


In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-21 19:05:27,429 : INFO : common  : created directory at: artifacts ]
[2025-08-21 19:05:27,430 : INFO : common  : created directory at: artifacts/data_ingestion ]
[2025-08-21 19:05:27,431 : INFO : 4037744573  : File already exists of size: ~ 11345 KB ]


In [ ]:
config = ConfigurationManager()

In [ ]:

yaml_file = PARAMS_FILE_PATH

In [ ]:
yaml_file 

In [ ]:
import yaml
with open(yaml_file, "r") as f:
    content = yaml.safe_load(f)
print(f"DEBUG: YAML content = {content}")

In [ ]:
type(content)